# Overview

This notebook continues the exercise of predicting car prices based on Craigslist listings.

This notebook uses `pandas` and `scikit-learn` two popular Python packages for data science. 

In order to make the processing more efficient, a sample of the data has been taken.


## Setup

Here are the packages that need to be loaded to run this example

These is where all the needed packages are imported for the exercise. If you get an `ModuleNotFoundError` then install the package (pip or conda) before continuing.These is where all the needed packages are imported for the exercise. If you get an `ModuleNotFoundError` then install the package (pip or conda) before continuing.

In [2]:
import time
import feather
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression, LassoLars, ElasticNet, Ridge, HuberRegressor, PassiveAggressiveRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import explained_variance_score, mean_squared_error

# Code from Introduction to Data Science Exercise

This code was used to create the profiles you used in earlier course in this series with the exception that this code is running on a 10% sample instead of the 500k+ listings.

## Read Sample Data from CSV to Pandas dataFrame

In [4]:
cars = feather.read_dataframe('../data/vehicles_samp.feather')

## Print first observations for sanity

In [5]:
cars.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,drive,size,type,paint_color,image_url,description,county,state,lat,long
0,7092558387,https://houston.craigslist.org/ctd/d/houston-2...,houston,https://houston.craigslist.org,16583,2012.0,infiniti,qx56,None,8 cylinders,...,rwd,None,SUV,white,https://images.craigslist.org/01515_WqeZnzBK5i...,Use this link to see more information on this ...,NaN,tx,29.7845,-95.6562
1,7092434302,https://chicago.craigslist.org/chc/cto/d/jolie...,chicago,https://chicago.craigslist.org,14500,2010.0,None,international prostar,None,None,...,None,None,None,None,https://images.craigslist.org/00H0H_aa1n1IOq1m...,2010 INTERNATIONAL PROSTAR MOTOR CUMMINS ISX ...,NaN,il,41.5378,-88.0572
2,7093160315,https://washingtondc.craigslist.org/doc/ctd/d/...,"washington, DC",https://washingtondc.craigslist.org,21995,2015.0,ford,f150,excellent,6 cylinders,...,4wd,None,pickup,red,https://images.craigslist.org/00101_iRbIxt72oH...,Drive it home today. Call (Or Text) us now !!C...,NaN,dc,39.1858,-78.1827
3,7092772132,https://bham.craigslist.org/cto/d/birmingham-2...,birmingham,https://bham.craigslist.org,3000,2004.0,ford,expedition xlt,good,8 cylinders,...,fwd,full-size,SUV,brown,https://images.craigslist.org/00k0k_1UF9dtsc3r...,Great work truck. Lots of road miles... We w...,NaN,al,33.4995,-86.7268
4,7085813538,https://ocala.craigslist.org/ctd/d/atlanta-201...,ocala,https://ocala.craigslist.org,18000,2019.0,kia,optima lx sedan 4d,good,None,...,fwd,None,sedan,silver,https://images.craigslist.org/00505_cjsO7sBEzL...,Welcome to Carvana—the new way to buy a car (e...,NaN,fl,33.7792,-84.4118


## Print the columns and their respective types for the cars dataFrame

In [7]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53976 entries, 0 to 53975
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            53976 non-null  int64  
 1   url           53976 non-null  object 
 2   region        53976 non-null  object 
 3   region_url    53976 non-null  object 
 4   price         53976 non-null  int64  
 5   year          53888 non-null  float64
 6   manufacturer  51622 non-null  object 
 7   model         53187 non-null  object 
 8   condition     30499 non-null  object 
 9   cylinders     32189 non-null  object 
 10  fuel          53642 non-null  object 
 11  odometer      44124 non-null  float64
 12  title_status  53683 non-null  object 
 13  transmission  53587 non-null  object 
 14  vin           31467 non-null  object 
 15  drive         38361 non-null  object 
 16  size          16826 non-null  object 
 17  type          39360 non-null  object 
 18  paint_color   36692 non-nu

## Create profile of full dataset using `pandas-profile`

In [8]:
prof = ProfileReport(cars)
prof.to_file(output_file='vehicle_profile.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 88.49it/s]


## Filter the data for cars with more than 300,000 miles and sort decending

This is an example of how Python programers can chain methods to each other to accomplish multiple tasks in the same line of code

In [9]:
cars[cars['odometer']>300000].filter(items=['year','price','odometer','model']).sort_values('odometer', ascending=False).head()
#filter(items=['one', 'three'])
#,['year','price','odometer','model']]

,year,price,odometer,model
22587,1962.0,2000,10000000.0,180d
34185,1990.0,10500,9999999.0,1990 nternational
612,1999.0,700,9000000.0,cavalier
3942,2012.0,8500,7691300.0,journey
22613,2019.0,5000,4728224.0,silverado 1500


## Filter the number of columns

With wide datasets it is often hard to see all the columns. The code below is an example of how the columns can be filtered to just those that are desired.

As with all things Python, there are many ways to accomplish this task

In [10]:
cars.filter(items=['year','price','odometer','model']).sort_values('odometer', ascending=False).head()


,year,price,odometer,model
22587,1962.0,2000,10000000.0,180d
34185,1990.0,10500,9999999.0,1990 nternational
612,1999.0,700,9000000.0,cavalier
3942,2012.0,8500,7691300.0,journey
22613,2019.0,5000,4728224.0,silverado 1500


In [11]:
cars.filter(items=['year','price','odometer','model']).sort_values('price', ascending=False).head()

,year,price,odometer,model
11052,2005.0,3755744318,154000.0,f150 4 x 4
22414,2008.0,3567587327,119501.0,ranger super cab
1130,2017.0,3253587287,28986.0,silverado 1500 crew cab
26968,1955.0,123456789,NaN,NaN
30892,1955.0,123456789,NaN,NaN


# Create new dataFrame `cars2` 

`cars2` only includes listings that meet all the following criteria
*    newer than 2000
*    cost less than $40,000
*    have less than 100,000 miles

In [12]:
cars2 = cars[(cars['year']>2000) & (cars['price']<40000) & (cars['odometer']<100000)]

## Comparing the shape of `cars` and `cars2`

I want to see how many rows and columns `cars` has and how many were lost due to the `filter` method

The data shows a reduction from 540K to 219K

In [13]:
cars.shape

(53976, 25)

In [14]:
cars2.shape

(22002, 25)

In [15]:
prof = ProfileReport(cars2)
prof.to_file(output_file='vehicle_profile2.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 61.87it/s]


In [16]:
cars2.to_csv('../data/cars2.zip')

In [17]:
cars2 = pd.read_csv('../data/cars2.zip')

In [18]:
cars2.dtypes

Unnamed: 0        int64
id                int64
url              object
region           object
region_url       object
price             int64
year            float64
manufacturer     object
model            object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
vin              object
drive            object
size             object
type             object
paint_color      object
image_url        object
description      object
county          float64
state            object
lat             float64
long            float64
dtype: object

# Introduction to Modeling and Machine Learning

The rest of this notebook is work that should be completed along with the Modeling and Machine Learning course



# Setup for data prep and modeling

To make the code more portable and easier to update. In the cell below I will define the inputs and targets.

The inputs are the attributes of a car that will be used to predict the price, which is the target.

To deal with missing values, there are different treatments for nominal vs interval data. Below lists are defined to identify the type of each variable.

The virtue of this type of organization is that to add or subtract a variable from the model, it only requires modifications to this cell and then a rerun of the code below. 

It also allows more copy/paste from project to projects saving time in the long run.

In [9]:
inputs = cars2.drop(columns=['price','id','url','region_url','vin','description','county','image_url'])
nominal = ['region', 'manufacturer', 'fuel', 'model','title_status','transmission', 'drive', 'type','paint_color','state']
ordinal = ['condition','size','cylinders']
interval = ['year','odometer','lat','long']
target = cars2.price
print(inputs.shape, target.shape)


(218870, 17) (218870,)


## Create Pipelines

A `pipeline` is a process or set of steps that you want to organize into specific pattern.

In this pipeline below, each variable is imputed if it is missing based on its type and then scaled to improve model performance (scaling input is a best practice along with spliting data to training and validation).

In [10]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, interval),
        ('cat', categorical_transformer, nominal),
        ('ord', categorical_transformer, ordinal)])

## Split the data into training and validation (70/30)

In [11]:
input_train, input_test, target_train, target_test = train_test_split(inputs, target, test_size = 0.30, random_state=9878)

# Build the models

In the code below a list of many algorythnms is provided and for each a model is build on the training data. 

The model is then applied to the validation data and assessment metrics are calculated.

The assessment metrics are then printed so that comparisons can be made.

**Remember:** there is no single best modeling algorthym so you should try many types and parameterizations based on the business problems

In [62]:
# Creates a results DataFrame. This code is in a different cell to prevent loss run to run.
results_df = pd.DataFrame(columns=('model','r_2','explainedVariance','mse','buildTime','scoreTime'))
results_df['r_2'] = results_df['r_2'].map('${:,.4f}'.format)
results_df['explainedVariance'] = results_df['explainedVariance'].map('${:,.4f}'.format)
results_df['mse'] = results_df['mse'].map('${:,.0f}'.format)
results_df['buildTime'] = results_df['buildTime'].map('${:,.1f}'.format)
results_df['scoreTime'] = results_df['scoreTime'].map('${:,.1f}'.format)

In [67]:
%%time
# List of models to run
model_list = [LinearRegression(), ElasticNet(), Ridge(), HuberRegressor(), PassiveAggressiveRegressor(), DecisionTreeRegressor(), RandomForestRegressor(), GradientBoostingRegressor(), MLPRegressor()]
model_list = [MLPRegressor(early_stopping=True, hidden_layer_sizes=(3,50))]
# LassoLars(), 

# loop through model list
for model in model_list:
    build_model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)])
    ts_build = time.time()
    build_model.fit(input_train, target_train)
    ts_score = time.time()
    target_pred = build_model.predict(input_test)
    ts_done = time.time()
    results = pd.DataFrame([[model, 
                build_model.score(input_test, target_test), 
                explained_variance_score(target_test, target_pred), 
                mean_squared_error(target_test, target_pred), 
                ts_score-ts_build, 
                ts_done-ts_score]], columns=results_df.columns)
    # append results to dataFrame
    results_df = results_df.append(results, ignore_index=True)

    print("model score for {}: R_2: {:0.4f} Explained Variance:{:0.4f} MSE:{:0.0f} Build Time(s):{:0.1f} Score Time(s):{:0.1f}".format(model, build_model.score(input_test, target_test), explained_variance_score(target_test, target_pred), mean_squared_error(target_test, target_pred), ts_score-ts_build, ts_done-ts_score))

model score for MLPRegressor(): R_2: 0.6342 Explained Variance:0.6342 MSE:37016331 Build Time(s):19540.3 Score Time(s):2.1
CPU times: user 14h 42min 43s, sys: 2h 16min 48s, total: 16h 59min 31s
Wall time: 5h 25min 45s


In [78]:
#results_df['r_2'] = results_df['r_2'].map('{:0.4f}'.format)
#results_df['explainedVariance'] = results_df['explainedVariance'].map('{:0.4f}'.format)
#results_df['mse'] = results_df['mse'].map('{:,0.0f}'.format)
#results_df['buildTime'] = results_df['buildTime'].map('{:.1f}'.format)
#results_df['scoreTime'] = results_df['scoreTime'].map('{:.1f}'.format)
results_df

,model,r_2,explainedVariance,mse,buildTime,scoreTime
0,ElasticNet(),$0.2273,$0.2273,"$78,199,094",$21.1,$0.4
1,Ridge(),$0.5074,$0.5074,"$49,848,804",$3.6,$0.4
2,HuberRegressor(),$0.4090,$0.4201,"$59,806,606",$6.2,$0.4
3,ElasticNet(selection='random'),$0.2273,$0.2273,"$78,199,078",$64.4,$0.4
4,Ridge(),$0.5074,$0.5074,"$49,848,804",$3.7,$0.4
5,HuberRegressor(),$0.4090,$0.4201,"$59,806,606",$6.1,$0.4
6,MLPRegressor(),$0.6342,$0.6342,"$37,016,331","$19,540.3",$2.1


In [ ]:
scaler = StandardScaler()
scaler.fit(input_train.loc[interval])

onehot = OneHotEncoder(dtype=np.int, sparse=True)
onehot.fit(input_train.loc[nominal])

input_train_nominal = onehot.transform(input_train.loc[nominal])
input_train_interval = scaler.transform(input_train.loc[interval])
input_test = scaler.transform(input_test)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import cross_val_score



def displaymetrics(code: list, models: list, X_train: np.array, X_test: np.array, X: pd.DataFrame):
    for model in models:  # or for i in range(0, len(models)):
        y_score = model.fit(X_train, y_train).decision_function(X_test)
        # or y_score = models[i].fit(X_train, y_train).decision_function(X_test)
        fpr, tpr, _ = roc_curve(y_test, y_score)
        # etc etc